In [ ]:
! pip3 install confluent_kafka

## Producer - Write messages to a Kafka Topic


Configure the producer

In [ ]:
from confluent_kafka import Consumer, Producer, KafkaError
import psycopg2
import json

In [ ]:
conf = {
    'bootstrap.servers': 'b-1.monstercluster1.6xql65.c3.kafka.eu-west-2.amazonaws.com:9092',  # This is the address of your Kafka broker(s)
    'client.id': 'retail_store_producer_joe', # give the producer a name
    'acks': 1,
    'message.timeout.ms': 5000 
}

producer = Producer(conf)

In [ ]:
transaction_data = {
    'product_name': 'rocket_ship',
    'category': 'toy',
    'amount': 10
}

# Convert the data to string format; often JSON is used
import json
transaction_string = json.dumps(transaction_data)

Send message

In [ ]:
def delivery_report(err, msg):
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}] - Content: {}'.format(msg.topic(), msg.partition(), msg.value().decode('utf-8')))

# produce method takes the name of the topic to which we are sending the message, the message, and callback which returns a delivery report
producer.produce('retail_product_topic', transaction_string, callback=delivery_report)
producer.flush()

## Consumer

Collect messages we have just written to the kafka topic and insert into the DB
- Kafka topic is a log or sequenced list of events (messages) 

In [ ]:
PERSONAL_ID = "joe" # To ensure all users read the entire topic

conf = {
    'bootstrap.servers': 'b-1.monstercluster1.6xql65.c3.kafka.eu-west-2.amazonaws.com:9092',  # Address of your Kafka broker(s)
    'group.id': f'retail_store_group_{PERSONAL_ID}', #specify the kfafka consumer group (this group of consumers work to consumer messages from the kafka topic)
    'auto.offset.reset': 'earliest'  # Start reading from the beginning of the topic
}

# create kafka consumer instance
consumer = Consumer(conf)

consumer.subscribe(['retail_product_topic'])

# Connect to the PostgreSQL database
connection = psycopg2.connect(
    dbname="retaildb",
    user="ec2-user",
    password="your_password",
    host="localhost",         
    port="5432"
)
cursor = connection.cursor()

try:
    while True:
        msg = consumer.poll(1.0)
        
        if msg is None:
            continue

        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                print(f"Reached end of partition {msg.partition()}.")
            else:
                print(f"Error while consuming message: {msg.error()}")
        else:
            # Parse the message value
            product_data = json.loads(msg.value().decode('utf-8'))
            print(product_data)
            
            # Insert the data into the database
            insert_query = """
            INSERT INTO retail_products (product_name, product_price, product_category) 
            VALUES (%s, %s, %s)
            """
            cursor.execute(insert_query, (
                product_data["product_name"],
                product_data["amount"],
                product_data["category"]
            ))
            
            # Commit the transaction
            connection.commit()

except KeyboardInterrupt:
    pass
finally:
    cursor.close()
    connection.close()
    consumer.close()


Display all rows from the retail_products table

In [ ]:
# Connect to the PostgreSQL database
connection = psycopg2.connect(
    dbname="retaildb",
    user="ec2-user",
    password="your_password",
    host="localhost",
    port="5432"
)
cursor = connection.cursor()

try:
    # SELECT query to retrieve all rows from the table
    select_query = "SELECT * FROM retail_products"
    cursor.execute(select_query)

    # Fetch all rows from the result set
    rows = cursor.fetchall()

    # Print column headers
    col_names = [desc[0] for desc in cursor.description]
    print(" | ".join(col_names))

    # Print each row
    for row in rows:
        print(" | ".join(map(str, row)))

finally:
    cursor.close()
    connection.close()